In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV, KFold
from sklearn.feature_selection import f_regression, mutual_info_regression, f_classif
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


## Part I - Data Pre-Proccessing
- ### Read In Data and Define Final Feature Set
- ### deleted all race columns except minority patient binary flag and ethnicity_14 (hispanic)
- ### scale labs for both data frames 
- ### convert EMRGNCY and INOUT to binary 

In [17]:
#read in training data and test data
training_data = pd.read_csv('../Data/hip_fracture_training.csv')
test_data = pd.read_csv('../Data/hip_fracture_test.csv')

In [18]:
display(training_data.shape)
display(test_data.shape)

(29188, 219)

(7297, 219)

In [19]:
#final feature set based on the seperate models
full_cols = ['DeepSSIPost_81', 'OrganSpacePost_84', 'PneumoniaPost_88', 'ReintubPost_90', 'PulmEmbPost_92', 
    'ProgRenalPost_96', 'UTIPost_100', 'MIPost_105', 'BleedingTfnPost_108', 'DVTPost_109', 
    'DVTProphylaxisPost28_24','SepsisPost_112','ReOPPost_116', 'CardiacComplications', 
    'InfectiousComplications', 'Stroke_2', 'WBATPOD1_21', 'PostOpPRESSORE_26', 'PostOpDelirium_29', 
    'PRSODM', 'PRBUN', 'PRCREAT', 'PRWBC', 'PRHCT', 'PRPLATE', 
    'SmokePre_37', 'DyspneaPre_38', 'DyspneaPre_39', 'DyspneaPre_40', 
    'FncStatusPre_41', 'FncStatusPre_42', 'FncStatusPre_43', 'FncStatusPre_44', 
    'VentDepPre_46', 'COPDPre_48', 'AscitesPre_50', 'CHFPre_52', 'HTNPre_54', 
    'DialysisPre_56', 'CancerPre_58', 'WoundInfectionPre_60', 'SteroidPre_62', 
    'WtLossPre_64', 'BleedingDisPre_66', 'TfnPre_68', 'SepsisPre_69', 'SepsisPre_70', 
    'SepsisPre_71', 'SepsisPre_72', 'ASAPre_73', 'ASAPre_74', 'ASAPre_75', 'ASAPre_76', 
    'ASAPre_77', 'ASAPre_78', 'PreOpDementia_2', 'PreOpDelirium_5', 'PreOpBoneMeds_7', 
    'PreOpMOBAID_10', 'PreOpPressSore_12', 'PartialorFullFuncStatus', 'DiabetesBinary','MFI5', 
    'ObesityBinary', 'MinorityPatient', 'AGE', 'Ethnicity_14', 'GenderData_1', 'GenderData_2', 
    'GenderData_3', 'TransferStatus_17', 'TransferStatus_18','DispoLocationPost_28','DispoLocationPost_30', 'MEDMGMT_14', 
    'StandardHip_17', 'HipFractureType_44','HipFractureType_45', 'HipFractureType_46', 
    'HipFractureType_47', 'HipFractureType_48','PathologicHipFractures_51', 
    'INOUT', 'EMERGNCY', 'OPTIME', 'TOTHLOS', 'HTOODAY','READMISSION30D']

In [20]:
#define filter data function and filter BOTH training and test data frames
def filter_data(df_train, df_test, cols): 

    #filters for only the preoperative columns
    df_train_filtered = df_train[cols]
    df_test_filtered = df_test[cols]

    return df_train_filtered, df_test_filtered   

#call function to get 
training_data_final, test_data_final = filter_data(training_data, test_data, full_cols)

In [ ]:
#display information for training data
display(training_data_final.shape)
display(training_data_final.isna().sum())
display(training_data_final.info())

In [ ]:
#display information for test data
display(test_data_final.shape)
display(test_data_final.isna().sum())
display(test_data_final.info())


In [34]:
#next we need to scale the lab values 
lab_cols = ['PRSODM', 'PRBUN', 'PRCREAT', 'PRWBC', 'PRHCT', 'PRPLATE']

def scale_labs(training_df, test_df, lab_cols):

    # impute missing values with the median
    imputer = SimpleImputer(strategy='median')

    # create a DataFrame with only the lab columns
    training_lab_df = training_df[lab_cols]

    # replace missing values (-99) with NaN
    training_lab_df = training_lab_df.replace(-99, np.nan)

    #fill missing lab values in training data with median 
    training_lab_df = imputer.fit_transform(training_lab_df)

    #replace test df -99 with na in test data 
    test_lab_df = test_df[lab_cols].replace(-99, np.nan)

    #fill  missing lab values in test data with median lab values 
    test_lab_df = imputer.transform(test_lab_df)

    # create a StandardScaler object
    scaler = StandardScaler()

    # fit the scaler to the lab data
    scaler.fit(training_lab_df)

    # transform the lab data in training and test sets
    scaled_training_data = scaler.transform(training_lab_df)
    scaled_test_data = scaler.transform(test_lab_df)

    # create a new DataFrames with the scaled lab data
    scaled_training_data = pd.DataFrame(scaled_training_data, columns=lab_cols)
    scaled_test_data = pd.DataFrame(scaled_test_data, columns=lab_cols)

    # create a new DataFrame that contains both the scaled lab columns and the other columns from the input DataFrame
    scaled_training_data = pd.concat([training_df.drop(lab_cols, axis=1), scaled_training_data], axis=1)
    scaled_test_data = pd.concat([test_df.drop(lab_cols, axis=1), scaled_test_data], axis=1)

    return scaled_training_data, scaled_test_data


training_scaled, test_scaled = scale_labs(training_data_final, test_data_final, lab_cols)

In [36]:
#display scaled training data 
display(training_scaled.shape)
display(training_scaled.head())

(29188, 88)

,DeepSSIPost_81,OrganSpacePost_84,PneumoniaPost_88,ReintubPost_90,PulmEmbPost_92,ProgRenalPost_96,UTIPost_100,MIPost_105,BleedingTfnPost_108,DVTPost_109,DVTProphylaxisPost28_24,SepsisPost_112,ReOPPost_116,CardiacComplications,InfectiousComplications,Stroke_2,WBATPOD1_21,PostOpPRESSORE_26,PostOpDelirium_29,SmokePre_37,DyspneaPre_38,DyspneaPre_39,DyspneaPre_40,FncStatusPre_41,FncStatusPre_42,FncStatusPre_43,FncStatusPre_44,VentDepPre_46,COPDPre_48,AscitesPre_50,CHFPre_52,HTNPre_54,DialysisPre_56,CancerPre_58,WoundInfectionPre_60,SteroidPre_62,WtLossPre_64,BleedingDisPre_66,TfnPre_68,SepsisPre_69,SepsisPre_70,SepsisPre_71,SepsisPre_72,ASAPre_73,ASAPre_74,ASAPre_75,ASAPre_76,ASAPre_77,ASAPre_78,PreOpDementia_2,PreOpDelirium_5,PreOpBoneMeds_7,PreOpMOBAID_10,PreOpPressSore_12,PartialorFullFuncStatus,DiabetesBinary,MFI5,ObesityBinary,MinorityPatient,AGE,Ethnicity_14,GenderData_1,GenderData_2,GenderData_3,TransferStatus_17,TransferStatus_18,DispoLocationPost_28,DispoLocationPost_30,MEDMGMT_14,StandardHip_17,HipFractureType_44,HipFractureType_45,HipFractureType_46,HipFractureType_47,HipFractureType_48,PathologicHipFractures_51,INOUT,EMERGNCY,OPTIME,TOTHLOS,HTOODAY,READMISSION30D,PRSODM,PRBUN,PRCREAT,PRWBC,PRHCT,PRPLATE
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,73,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,Inpatient,No,101,5,1,0,0.240328,-0.457357,-0.502422,0.460396,0.306751,0.582291
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,90,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,Inpatient,No,38,6,0,0,0.240328,-1.126433,-0.338898,-0.140167,0.343554,-0.824676
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,90,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,Inpatient,Yes,71,4,0,0,-0.024477,-0.457357,-0.590474,-0.085570,0.914002,-0.694401
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,69,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Inpatient,No,165,7,1,0,-0.289283,-0.039186,0.038465,-0.931818,1.521253,0.347796
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,3,0,0,88,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,Inpatient,Yes,111,11,2,1,0.505134,-0.457357,-0.464686,0.624186,0.085932,0.425961


In [37]:
#display scaled test data 
display(test_scaled.shape)
display(test_scaled.head())


(7297, 88)

,DeepSSIPost_81,OrganSpacePost_84,PneumoniaPost_88,ReintubPost_90,PulmEmbPost_92,ProgRenalPost_96,UTIPost_100,MIPost_105,BleedingTfnPost_108,DVTPost_109,DVTProphylaxisPost28_24,SepsisPost_112,ReOPPost_116,CardiacComplications,InfectiousComplications,Stroke_2,WBATPOD1_21,PostOpPRESSORE_26,PostOpDelirium_29,SmokePre_37,DyspneaPre_38,DyspneaPre_39,DyspneaPre_40,FncStatusPre_41,FncStatusPre_42,FncStatusPre_43,FncStatusPre_44,VentDepPre_46,COPDPre_48,AscitesPre_50,CHFPre_52,HTNPre_54,DialysisPre_56,CancerPre_58,WoundInfectionPre_60,SteroidPre_62,WtLossPre_64,BleedingDisPre_66,TfnPre_68,SepsisPre_69,SepsisPre_70,SepsisPre_71,SepsisPre_72,ASAPre_73,ASAPre_74,ASAPre_75,ASAPre_76,ASAPre_77,ASAPre_78,PreOpDementia_2,PreOpDelirium_5,PreOpBoneMeds_7,PreOpMOBAID_10,PreOpPressSore_12,PartialorFullFuncStatus,DiabetesBinary,MFI5,ObesityBinary,MinorityPatient,AGE,Ethnicity_14,GenderData_1,GenderData_2,GenderData_3,TransferStatus_17,TransferStatus_18,DispoLocationPost_28,DispoLocationPost_30,MEDMGMT_14,StandardHip_17,HipFractureType_44,HipFractureType_45,HipFractureType_46,HipFractureType_47,HipFractureType_48,PathologicHipFractures_51,INOUT,EMERGNCY,OPTIME,TOTHLOS,HTOODAY,READMISSION30D,PRSODM,PRBUN,PRCREAT,PRWBC,PRHCT,PRPLATE
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,2,0,0,82,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,Inpatient,No,68,4,1,0,0.240328,-0.791895,-0.464686,-0.358554,-0.686933,0.269632
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,88,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,Inpatient,No,34,5,0,0,0.240328,0.880793,1.183132,4.473248,-0.944555,1.481186
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,87,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,Inpatient,Yes,89,3,0,0,-0.024477,-1.210067,-0.615631,-0.385852,0.729986,1.324856
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,85,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,Inpatient,Yes,103,4,0,0,-0.024477,0.713524,0.227146,0.596887,1.723670,0.608346
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,72,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,Inpatient,No,92,5,0,0,-0.818894,-0.624626,-0.716261,1.388538,0.490766,0.634401


In [38]:
#convert these two to binary - check value counts to make sure no new values in test data
display(test_scaled['EMERGNCY'].value_counts())
display(test_scaled['INOUT'].value_counts())

No     5990
Yes    1307
Name: EMERGNCY, dtype: int64

Inpatient     7270
Outpatient      27
Name: INOUT, dtype: int64

In [39]:
#convert EMERGENCY and INOUT to binary
def convert_binary(training_df, test_df): 

    #define binary encodings
    encoding_EMERGENCY = {'No': 0, 'Yes': 1}
    encoding_INOUT = {'Inpatient': 1, 'Outpatient': 0}

    #convert EMERGNCY and INOUT to binary in training set
    training_df['EMERGNCY'] = training_df['EMERGNCY'].map(encoding_EMERGENCY)
    training_df['INOUT'] = training_df['INOUT'].map(encoding_INOUT)
    training_df_encoded = training_df 

    #convert EMERGNCY and INOUT to binary in test set
    test_df['EMERGNCY'] = test_df['EMERGNCY'].map(encoding_EMERGENCY)
    test_df['INOUT'] = test_df['INOUT'].map(encoding_INOUT)
    test_df_encoded = test_df 

    return training_df_encoded, test_df_encoded

training_encoded, test_encoded = convert_binary(training_scaled, test_scaled)


In [40]:
display(training_encoded.shape)
display(training_encoded.head())

(29188, 88)

,DeepSSIPost_81,OrganSpacePost_84,PneumoniaPost_88,ReintubPost_90,PulmEmbPost_92,ProgRenalPost_96,UTIPost_100,MIPost_105,BleedingTfnPost_108,DVTPost_109,DVTProphylaxisPost28_24,SepsisPost_112,ReOPPost_116,CardiacComplications,InfectiousComplications,Stroke_2,WBATPOD1_21,PostOpPRESSORE_26,PostOpDelirium_29,SmokePre_37,DyspneaPre_38,DyspneaPre_39,DyspneaPre_40,FncStatusPre_41,FncStatusPre_42,FncStatusPre_43,FncStatusPre_44,VentDepPre_46,COPDPre_48,AscitesPre_50,CHFPre_52,HTNPre_54,DialysisPre_56,CancerPre_58,WoundInfectionPre_60,SteroidPre_62,WtLossPre_64,BleedingDisPre_66,TfnPre_68,SepsisPre_69,SepsisPre_70,SepsisPre_71,SepsisPre_72,ASAPre_73,ASAPre_74,ASAPre_75,ASAPre_76,ASAPre_77,ASAPre_78,PreOpDementia_2,PreOpDelirium_5,PreOpBoneMeds_7,PreOpMOBAID_10,PreOpPressSore_12,PartialorFullFuncStatus,DiabetesBinary,MFI5,ObesityBinary,MinorityPatient,AGE,Ethnicity_14,GenderData_1,GenderData_2,GenderData_3,TransferStatus_17,TransferStatus_18,DispoLocationPost_28,DispoLocationPost_30,MEDMGMT_14,StandardHip_17,HipFractureType_44,HipFractureType_45,HipFractureType_46,HipFractureType_47,HipFractureType_48,PathologicHipFractures_51,INOUT,EMERGNCY,OPTIME,TOTHLOS,HTOODAY,READMISSION30D,PRSODM,PRBUN,PRCREAT,PRWBC,PRHCT,PRPLATE
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,73,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,101,5,1,0,0.240328,-0.457357,-0.502422,0.460396,0.306751,0.582291
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,90,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,38,6,0,0,0.240328,-1.126433,-0.338898,-0.140167,0.343554,-0.824676
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,90,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,1,1,71,4,0,0,-0.024477,-0.457357,-0.590474,-0.085570,0.914002,-0.694401
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,69,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,165,7,1,0,-0.289283,-0.039186,0.038465,-0.931818,1.521253,0.347796
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,3,0,0,88,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,111,11,2,1,0.505134,-0.457357,-0.464686,0.624186,0.085932,0.425961


In [41]:
display(test_encoded.shape)
display(test_encoded.head())


(7297, 88)

,DeepSSIPost_81,OrganSpacePost_84,PneumoniaPost_88,ReintubPost_90,PulmEmbPost_92,ProgRenalPost_96,UTIPost_100,MIPost_105,BleedingTfnPost_108,DVTPost_109,DVTProphylaxisPost28_24,SepsisPost_112,ReOPPost_116,CardiacComplications,InfectiousComplications,Stroke_2,WBATPOD1_21,PostOpPRESSORE_26,PostOpDelirium_29,SmokePre_37,DyspneaPre_38,DyspneaPre_39,DyspneaPre_40,FncStatusPre_41,FncStatusPre_42,FncStatusPre_43,FncStatusPre_44,VentDepPre_46,COPDPre_48,AscitesPre_50,CHFPre_52,HTNPre_54,DialysisPre_56,CancerPre_58,WoundInfectionPre_60,SteroidPre_62,WtLossPre_64,BleedingDisPre_66,TfnPre_68,SepsisPre_69,SepsisPre_70,SepsisPre_71,SepsisPre_72,ASAPre_73,ASAPre_74,ASAPre_75,ASAPre_76,ASAPre_77,ASAPre_78,PreOpDementia_2,PreOpDelirium_5,PreOpBoneMeds_7,PreOpMOBAID_10,PreOpPressSore_12,PartialorFullFuncStatus,DiabetesBinary,MFI5,ObesityBinary,MinorityPatient,AGE,Ethnicity_14,GenderData_1,GenderData_2,GenderData_3,TransferStatus_17,TransferStatus_18,DispoLocationPost_28,DispoLocationPost_30,MEDMGMT_14,StandardHip_17,HipFractureType_44,HipFractureType_45,HipFractureType_46,HipFractureType_47,HipFractureType_48,PathologicHipFractures_51,INOUT,EMERGNCY,OPTIME,TOTHLOS,HTOODAY,READMISSION30D,PRSODM,PRBUN,PRCREAT,PRWBC,PRHCT,PRPLATE
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,2,0,0,82,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,68,4,1,0,0.240328,-0.791895,-0.464686,-0.358554,-0.686933,0.269632
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,88,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,34,5,0,0,0.240328,0.880793,1.183132,4.473248,-0.944555,1.481186
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,87,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,89,3,0,0,-0.024477,-1.210067,-0.615631,-0.385852,0.729986,1.324856
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,85,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,1,103,4,0,0,-0.024477,0.713524,0.227146,0.596887,1.723670,0.608346
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,72,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,92,5,0,0,-0.818894,-0.624626,-0.716261,1.388538,0.490766,0.634401


## Part II - Modelling 
- ### first we will define our labels and predictors for our training and test set
- ### then we will train a random forest using our hyperparameters from our optimized model and predict on our test set
- ### finally we will look at the feature importance and AUROC of our final model

In [49]:
#check for differences in class balance between training and test now - should be similar because split was stratified 
display(test_encoded['READMISSION30D'].value_counts())
display(training_encoded['READMISSION30D'].value_counts())

0    6570
1     727
Name: READMISSION30D, dtype: int64

0    26385
1     2803
Name: READMISSION30D, dtype: int64

In [54]:
#define split function
def split_data(training_encoded, test_encoded):
    X_train = training_encoded.drop(labels = ['READMISSION30D'], axis = 1)
    y_train = training_encoded['READMISSION30D']
    X_test = test_encoded.drop(labels = ['READMISSION30D'], axis = 1)
    y_test = test_encoded['READMISSION30D']

    return X_train,X_test,y_train,y_test

In [ ]:
#get label and predictors for training/test
X_train,X_test,y_train,y_test = split_data(training_encoded, test_encoded)

display(X_train.info())
display(X_test.info())

display(y_train.info())
display(y_test.info())

In [ ]:
#now we will 